## Triển khai lại simple NN trên MNIST để classifi số

In [21]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
import numpy as np
import sys
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/mnist2')

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# hyper parameters
num_epochs = 1
learning_rate = 0.001
batch_size = 64

# preparing data
mean = (0.5)
std = (0.27)
imagesTransform =transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

train_dataset = datasets.MNIST(root="../../data", download=False, train=True,
                               transform=imagesTransform)
test_dataset = datasets.MNIST(root="../../data", download=False, train=False,
                               transform=imagesTransform)

train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(dataset=test_dataset, shuffle=False, batch_size=batch_size)

print(train_dataset.classes)

# Visualize example
train_iter = iter(train_loader)
images, labels = train_iter.next()
print(images.shape)

def imshow(images):
    images = images /2 + 0.5
    npimg = images.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()
# imshow(torchvision.utils.make_grid(images))
writer.add_image('mnist_images',torchvision.utils.make_grid(images))

# Setup NN
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, images):
        outputs = self.linear1(images)
        outputs = self.relu(outputs)
        outputs = self.linear2(outputs)
        
        return outputs

input_size = 28*28
hidden_size = 100
output_size = 10

# model
model = NeuralNet(input_size, hidden_size, output_size).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
writer.add_graph(model, images.reshape(-1, 28*28))

n_total_steps = len(train_loader)
# training model
for epoch in range(num_epochs):
    n_correct = 0
    n_samples = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # accuracy
        _,preds = torch.max(outputs, 1)
        n_correct += torch.sum(preds == labels)
        n_samples += labels.shape[0]
        
        if( i+ 1) %100 == 0:
            writer.add_scalar('training loss: ',loss,epoch*n_total_steps + i)
            writer.add_scalar('accuracy', (n_correct / n_samples),epoch*n_total_steps + i)
    
    
    print(f"Epoch {epoch} / {num_epochs}, loss: {loss:.2f}, accuracy: {(n_correct / n_samples):.2f}")

# make pr curve tensorboard
tensorboard_labels = []
tensorboard_preds = []


# make predict
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i,(images, labels) in enumerate(test_loader):
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
    
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct  += torch.sum(preds == labels)

        # add data to tensorboard pr_curve
        class_predictions = [nn.functional.softmax(output, dim = 0) for output in outputs]
        tensorboard_labels.append(preds)
        tensorboard_preds.append(class_predictions)
    tensorboard_preds = torch.cat([torch.stack(batch) for batch in tensorboard_preds])
    tensorboard_labels = torch.cat(tensorboard_labels)
    acc = 100.0*n_correct / n_samples
    print(f'accuracy = {acc}')
    
    classes = range(10)
    for i in classes:
        labels_i = tensorboard_labels == i
        preds_i = tensorboard_preds[:, i]
        writer.add_pr_curve(str(i), labels_i, preds_i, global_step=0)
        writer.close()

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
torch.Size([64, 1, 28, 28])
Epoch 0 / 1, loss: 0.05, accuracy: 0.89
accuracy = 93.19999694824219
